In [29]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns

In [30]:
import zipfile

In [31]:
#Öppnar zipfilen
zip_file_path = 'ml-latest.zip' 
extract_folder = 'Labb_1' 
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [34]:
df_movies = pd.read_csv("Data/ml-latest/movies.csv", index_col = 0)
df_movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [35]:
df_ratings = pd.read_csv("Data/ml-latest/ratings.csv", index_col = 0)
df_ratings.head()

,movieId,rating,timestamp
userId,,,
1,1,4.0,1225734739
1,110,4.0,1225865086
1,158,4.0,1225733503
1,260,4.5,1225735204
1,356,5.0,1225735119


## Movielens full - EDA